# Combined Proteomic Comparison 


## Step 1: Library Imports

Run this cell to import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
#sys.path.append('C:\\Users\\brittany henderson\\GitHub\\GBM_for_CPTAC\\')
#import cis_functions as f

import cptac
import cptac.utils as u

In [2]:
def add_significance_col(results_df, num_comparisons):
    "bonferroni multiple hypothesis"""
    alpha = .05
    bonferroni_cutoff = alpha / num_comparisons
    
    pval = results_df['P_Value']
    if float(pval[0]) <= bonferroni_cutoff:
        results_df['Significant'] = True
    else: 
        results_df['Significant'] = False
    return results_df

def wrap_ttest_return_all(df, label_column, comparison_columns, total_tests, alpha=.05):
    try:
        #Verify precondition that label column exists and has exactly 2 unique values
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        #Partition dataframe into two sets, one for each of the two unique values from the label column
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]
        
        #Determine the number of real valued columns on which we will do t-tests
        #sites = len(comparison_columns.columns)
        number_of_comparisons = total_tests # ? phospho sites or num freq mut genes doing cis comp
        
        #Use a bonferroni correction to adjust for multiple testing by altering the p-value needed for acceptance
        bonferroni_cutoff = alpha/number_of_comparisons
        
        #Store all comparisons with their p-values in a dictionary
        all_comparisons = {}
        
        #Loop through each comparison column, perform the t-test, and determine whether it meets the significance cutoff'''
        for column in comparison_columns:
            stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
            all_comparisons[column] = pval
    
        #Sort dictionary to list smallest p-values first
        sorted_comparisons = sorted(all_comparisons.items(), key=lambda kv: kv[1])
        #Format as a dataframe and return to caller
        all_comparisons_df = pd.DataFrame.from_dict(sorted_comparisons)
        all_comparisons_df.columns = ['Comparison', 'P_Value']
        
                                               
        all_comparisons_sig_col = add_significance_col(all_comparisons_df, number_of_comparisons)
        return all_comparisons_sig_col
                                
    except:
        print("Incorrectly Formatted Dataframe!")
        return None


In [3]:
# note: pass reduced omics # CHECK would include non coding, but none in fm genes

def format_mutated_wt_comparison(reduced_omics_df):
    # Step 2 - Create binary column 
    reduced_omics_df['binary_mutations'] = np.where(
                reduced_omics_df[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
    tumors = reduced_omics_df[reduced_omics_df.Sample_Status == 'Tumor'] #drop Normal samples
    columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
    mut_status = tumors.drop(columns_to_drop, axis = 1)

    return mut_status

In [4]:
def get_missense_truncation_comparison(cancer_object, reduced_omics_df):
    #get omics data and tumors
    tumors = reduced_omics_df.loc[reduced_omics_df['Sample_Status'] == 'Tumor'] #drop Normal samples


    somatic_mutations = cancer_object.get_somatic_mutation().reset_index()

    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'Truncation', 
            'frameshift deletion': 'Truncation', 'frameshift insertion': 'Truncation', 
            'stopgain': 'Truncation', 'stoploss': 'Truncation', 'nonsynonymous SNV': 'Missense',
            'nonframeshift insertion': 'Missense','nonframeshift deletion': 'Missense', 
            'nonframeshift substitution': 'Missense'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'Missense', 'In_Frame_Ins': 'Missense',
            'Missense_Mutation': 'Missense', 'Frame_Shift_Del': 'Truncation','Nonsense_Mutation': 'Truncation', 
            'Splice_Site': 'Truncation', 'Frame_Shift_Ins': 'Truncation','Nonstop_Mutation':'Truncation'}

    mutations_replaced_M_T = somatic_mutations.replace(missence_truncation_groups)
    mutations_replaced_M_T = mutations_replaced_M_T.loc[mutations_replaced_M_T['Gene'] == gene]

    # group mutation categories
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'Missense']
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'Truncation']

    #get lists of unique samples for missence and trucation categories
    miss_unique_samples = list(miss['Sample_ID'].unique())
    trunc_unique_samples = list(trunc['Sample_ID'].unique())
    
    #check if there is only one type of mutation for the specific gene
    if miss_unique_samples == []:
        print('Only truncation type mutations found for', gene+'.', 
             'Not possible to compare missense with wildtype.')
        truncation_omics = tumors.loc[tumors.index.isin(trunc_unique_samples)]
        truncation_omics = truncation_omics.assign(binary_mutations = 'Truncation')
        columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
        binary_mut_omics = truncation_omics.drop(columns_to_drop, axis = 1)
        return binary_mut_omics
    elif trunc_unique_samples == []:
        print('Only missence type mutations found for', gene+'.', 
             'Not possible to compare truncation with wildtype.')
        missence_omics = tumors.loc[tumors.index.isin(miss_unique_samples)]
        missence_omics = missence_omics.assign(binary_mutations = 'Missense')
        columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
        binary_mut_omics = missence_omics.drop(columns_to_drop, axis = 1)
        return binary_mut_omics

    ### Step 2 - Create the binary column needed to do the comparison
    # Get mutation catagories with omics data
    missence_omics = tumors.loc[tumors.index.isin(miss_unique_samples)]
    missence_omics = missence_omics.assign(binary_mutations = 'Missense')
    truncation_omics = tumors.loc[tumors.index.isin(trunc_unique_samples)]
    truncation_omics = truncation_omics.assign(binary_mutations = 'Truncation')
    binary_mut_omics = missence_omics.append(truncation_omics)

    # Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
    columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
    binary_mut_omics = binary_mut_omics.drop(columns_to_drop, axis = 1)

    return binary_mut_omics


In [5]:
def format_pval_annotation(pval_symbol, x1, x2, line_start = .05, line_height=.05):
    # for manual adjustment to pval annotations
    
    y, h = line_start, line_height
    plt.plot([x1, x1, x2, x2], #draw horizontal line
             [y, y+h, y+h, y], #vertical line
             lw=1.5, color= '.3')
    plt.text((x1+x2)*.5, # half between x coord
             y+h, pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")

# used in pval_annotation
def get_pval_symbol(pval_df):
    if pval_df.iloc[0:,2].bool() == True:
        pval_symbol = '*'
    else:
        pval_symbol = 'ns'
    return pval_symbol

def pval_annotation(pval_df, plotted_df):
    # pval symbol
    num_pvals = len(pval_df)
    omics_col = plotted_df.columns[0]
    

    # annotation for Mutated and Wiltype
    if num_pvals == 1:
        mut_pval = pval_df.loc[pval_df.index == "Mutated/Wildtype"]
        mut_pval_symbol = get_pval_symbol(pval_df)
        
        x1, x2 = 0, 1   # columns (first column: 0, see plt.xticks())
        y, h = plotted_df[omics_col].max() + .05, .05     
        plt.plot([x1, x1, x2, x2], #draw horizontal line
                 [y, y+h, y+h, y], #vertical line
                 lw=1.5, color= '.3')
        plt.text((x1+x2)*.5, # half between x coord
                 y+h, mut_pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")
    
    # annotation for Missense, Truncation and Wildtype
    elif num_pvals == 2:
        # Missense
        miss_pval = pval_df.loc[pval_df.index == "Missense/Wildtype"]
        miss_pval_symbol = get_pval_symbol(miss_pval)
    
        x1, x2 = 0, 1   # columns (first column: 0, see plt.xticks())
        y, h = plotted_df[omics_col].max() + .05, .05     
        plt.plot([x1, x1, x2, x2], #draw horizontal line
                 [y, y+h, y+h, y], #vertical line
                 lw=1.5, color= '.3')
        plt.text((x1+x2)*.5, # half between x coord
                 y+h, miss_pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")
        
        # Truncation 
        trunc_pval = pval_df.loc[pval_df.index == "Truncation/Wildtype"]
        trunc_pval_symbol = get_pval_symbol(trunc_pval)
        
        x3, x4 = 0, 2   # columns (first column: 0, see plt.xticks())
        y2, h2 = plotted_df[omics_col].max() + .15, .05     
        plt.plot([x3, x3, x4, x4], #draw horizontal line
                 [y2, y2+h2, y2+h2, y2], #vertical line
                 lw=1.5, color= '.3')
        plt.text((x3+x4)*.5, # half between x coord
                 y2+h2, trunc_pval_symbol, horizontalalignment='center', verticalalignment='bottom', color = "black")


## Step 2: Find the mutation frequency of the gene of interest

In [6]:
#cptac.download(dataset='ccrcc', version='0.0')
brain = cptac.Gbm()
desired_cutoff = 0.05
gene = 'RB1'

In [7]:
#FIX
#freq_mut = u.get_frequently_mutated(brain, cutoff = desired_cutoff)
#freq_mut.loc[freq_mut['Gene'] == gene]

## Step 3: cis comparisons for omics 

Determine if the DNA mutation has an effect on the omics measurement. In order to do this, we have a few steps in code. These steps are found in the format_cis_comparison_data function.
1. get a table with both the omics and mutation data for tumors
2. get a binary column from the mutation data to separate our samples
3. format data frame to be used in the T-test
4. send data to the T-test.

The format_cis_comparison_data does the first 3 steps (for dataframes with multi-indexes).

# Proteomics RB1 cis comparison

Proteomics: Mutated and Wildtype rb1

In [8]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "RB1") # single col level


# Step 2 & 3 - Get binary column and format
RB1_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
RB1_prot_mut_wt


Name,RB1_proteomics,binary_mutations
Sample_ID,,
S001,-1.799180,Mutated
S002,-1.071307,Mutated
S003,0.154265,Wildtype
S004,-0.884636,Mutated
S005,0.041888,Wildtype
...,...,...
S100,-0.210103,Wildtype
S101,1.331462,Wildtype
S102,-0.003929,Wildtype


In [9]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["RB1"+'_proteomics']
prot_col = "RB1"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(RB1_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,RB1_proteomics,1.139144e-08,True


Proteomics: Missense and Truncation compared to wildtype

In [10]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = RB1_prot_mut_wt.loc[RB1_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
RB1_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = RB1_prot_miss_trunc_wildtype.loc[RB1_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

prot_results_miss_trunc = prot_results_trunc
prot_results_miss_trunc
RB1_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,RB1_proteomics,binary_mutations
Sample_ID,,
S001,-1.799180,Truncation
S002,-1.071307,Truncation
S004,-0.884636,Truncation
S014,-1.221436,Truncation
S041,-0.718502,Truncation
...,...,...
S100,-0.210103,Wildtype
S101,1.331462,Wildtype
S102,-0.003929,Wildtype


# CDK2 

In [11]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "CDK2") # single col level


# Step 2 & 3 - Get binary column and format
CDK2_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
CDK2_prot_mut_wt



Name,CDK2_proteomics,binary_mutations
Sample_ID,,
S001,0.507714,Mutated
S002,0.764774,Mutated
S003,0.415454,Wildtype
S004,0.157479,Mutated
S005,-0.423898,Wildtype
...,...,...
S100,0.386990,Wildtype
S101,0.678933,Wildtype
S102,-0.089902,Wildtype


In [12]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CDK2"+'_proteomics']
prot_col = "CDK2"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(CDK2_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,CDK2_proteomics,0.000005,True


In [13]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = CDK2_prot_mut_wt.loc[CDK2_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
CDK2_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = CDK2_prot_miss_trunc_wildtype.loc[CDK2_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

CDK2_prot_results_miss_trunc = prot_results_trunc
CDK2_prot_results_miss_trunc
CDK2_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,CDK2_proteomics,binary_mutations
Sample_ID,,
S001,0.507714,Truncation
S002,0.764774,Truncation
S004,0.157479,Truncation
S014,1.234162,Truncation
S041,0.149800,Truncation
...,...,...
S100,0.386990,Wildtype
S101,0.678933,Wildtype
S102,-0.089902,Wildtype


# CDK6

In [14]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "CDK6") # single col level


# Step 2 & 3 - Get binary column and format
CDK6_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
CDK6_prot_mut_wt


Name,CDK6_proteomics,binary_mutations
Sample_ID,,
S001,-1.011444,Mutated
S002,-0.907507,Mutated
S003,0.080839,Wildtype
S004,-1.560394,Mutated
S005,-0.964315,Wildtype
...,...,...
S100,0.744616,Wildtype
S101,1.802314,Wildtype
S102,0.354357,Wildtype


In [15]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CDK6"+'_proteomics']
prot_col = "CDK6"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(CDK6_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,CDK6_proteomics,0.000013,True


In [16]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = CDK6_prot_mut_wt.loc[CDK6_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
CDK6_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = CDK6_prot_miss_trunc_wildtype.loc[CDK6_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

CDK6_prot_results_miss_trunc = prot_results_trunc
CDK6_prot_results_miss_trunc
CDK6_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,CDK6_proteomics,binary_mutations
Sample_ID,,
S001,-1.011444,Truncation
S002,-0.907507,Truncation
S004,-1.560394,Truncation
S014,-0.686852,Truncation
S041,-0.797592,Truncation
...,...,...
S100,0.744616,Wildtype
S101,1.802314,Wildtype
S102,0.354357,Wildtype


# CDKN2A

In [17]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "CDKN2A") # single col level


# Step 2 & 3 - Get binary column and format
CDKN2A_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
CDKN2A_prot_mut_wt

Name,CDKN2A_proteomics,binary_mutations
Sample_ID,,
S001,1.249326,Mutated
S002,1.042734,Mutated
S003,0.046073,Wildtype
S004,1.275257,Mutated
S005,1.482385,Wildtype
...,...,...
S100,0.444150,Wildtype
S101,-0.515793,Wildtype
S102,-0.486051,Wildtype


In [18]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CDKN2A"+'_proteomics']
prot_col = "CDKN2A"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(CDKN2A_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,CDKN2A_proteomics,0.000496,True


In [19]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = CDKN2A_prot_mut_wt.loc[CDK6_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
CDKN2A_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = CDKN2A_prot_miss_trunc_wildtype.loc[CDK6_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

CDKN2A_prot_results_miss_trunc = prot_results_trunc
CDKN2A_prot_results_miss_trunc
CDKN2A_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,CDKN2A_proteomics,binary_mutations
Sample_ID,,
S001,1.249326,Truncation
S002,1.042734,Truncation
S004,1.275257,Truncation
S014,-0.596999,Truncation
S041,1.487689,Truncation
...,...,...
S100,0.444150,Wildtype
S101,-0.515793,Wildtype
S102,-0.486051,Wildtype


# TFDP1

In [20]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "TFDP1") # single col level


# Step 2 & 3 - Get binary column and format
TFDP1_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
TFDP1_prot_mut_wt

Name,TFDP1_proteomics,binary_mutations
Sample_ID,,
S001,-0.020599,Mutated
S002,0.070025,Mutated
S003,0.684527,Wildtype
S004,0.292072,Mutated
S005,-0.215651,Wildtype
...,...,...
S100,0.391004,Wildtype
S101,-0.186174,Wildtype
S102,-0.082127,Wildtype


In [21]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["TFDP1"+'_proteomics']
prot_col = "TFDP1"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(TFDP1_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,TFDP1_proteomics,0.003648,False


In [22]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = TFDP1_prot_mut_wt.loc[TFDP1_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
TFDP1_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = TFDP1_prot_miss_trunc_wildtype.loc[TFDP1_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

TFDP1_prot_results_miss_trunc = prot_results_trunc
TFDP1_prot_results_miss_trunc
TFDP1_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,TFDP1_proteomics,binary_mutations
Sample_ID,,
S001,-0.020599,Truncation
S002,0.070025,Truncation
S004,0.292072,Truncation
S014,0.881052,Truncation
S041,-0.196130,Truncation
...,...,...
S100,0.391004,Wildtype
S101,-0.186174,Wildtype
S102,-0.082127,Wildtype


# MORF4L2

In [23]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "MORF4L2") # single col level


# Step 2 & 3 - Get binary column and format
MORF4L2_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
MORF4L2_prot_mut_wt

Name,MORF4L2_proteomics,binary_mutations
Sample_ID,,
S001,0.665620,Mutated
S002,0.067084,Mutated
S003,0.018001,Wildtype
S004,0.197339,Mutated
S005,0.227773,Wildtype
...,...,...
S100,0.299612,Wildtype
S101,0.073286,Wildtype
S102,-0.272625,Wildtype


In [24]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["MORF4L2"+'_proteomics']
prot_col = "MORF4L2"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(MORF4L2_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,MORF4L2_proteomics,0.000281,True


In [25]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = MORF4L2_prot_mut_wt.loc[MORF4L2_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
MORF4L2_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = MORF4L2_prot_miss_trunc_wildtype.loc[MORF4L2_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

MORF4L2_prot_results_miss_trunc = prot_results_trunc
MORF4L2_prot_results_miss_trunc
MORF4L2_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,MORF4L2_proteomics,binary_mutations
Sample_ID,,
S001,0.665620,Truncation
S002,0.067084,Truncation
S004,0.197339,Truncation
S014,0.805755,Truncation
S041,0.553495,Truncation
...,...,...
S100,0.299612,Wildtype
S101,0.073286,Wildtype
S102,-0.272625,Wildtype


# CCND2

In [26]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "CCND2") # single col level


# Step 2 & 3 - Get binary column and format
CCND2_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
CCND2_prot_mut_wt

Name,CCND2_proteomics,binary_mutations
Sample_ID,,
S001,0.020714,Mutated
S002,0.041569,Mutated
S003,0.425060,Wildtype
S004,0.057872,Mutated
S005,1.150136,Wildtype
...,...,...
S100,-0.217383,Wildtype
S101,-0.151934,Wildtype
S102,0.107436,Wildtype


In [27]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CCND2"+'_proteomics']
prot_col = "CCND2"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(CCND2_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,CCND2_proteomics,0.00239,True


In [28]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = CCND2_prot_mut_wt.loc[CCND2_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
CCND2_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = CCND2_prot_miss_trunc_wildtype.loc[CCND2_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

CCND2_prot_results_miss_trunc = prot_results_trunc
CCND2_prot_results_miss_trunc
CCND2_prot_miss_trunc_wildtype

Only truncation type mutations found for RB1. Not possible to compare missense with wildtype.
Doing t-test comparison for truncation and wildtype


Name,CCND2_proteomics,binary_mutations
Sample_ID,,
S001,0.020714,Truncation
S002,0.041569,Truncation
S004,0.057872,Truncation
S014,-1.026672,Truncation
S041,NaN,Truncation
...,...,...
S100,-0.217383,Wildtype
S101,-0.151934,Wildtype
S102,0.107436,Wildtype


# TP53

In [29]:
brain2 = cptac.Gbm()
gene = "TP53"
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain2.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "TP53") # single col level


# Step 2 & 3 - Get binary column and format
TP53_prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
TP53_prot_mut_wt

Name,TP53_proteomics,binary_mutations
Sample_ID,,
S001,0.044573,Mutated
S002,0.117856,Mutated
S003,-0.120887,Wildtype
S004,1.832557,Mutated
S005,-0.288708,Wildtype
...,...,...
S100,-0.343235,Wildtype
S101,NaN,Wildtype
S102,NaN,Wildtype


In [30]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["TP53"+'_proteomics']
prot_col = "TP53"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(TP53_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

Doing t-test comparison for mutated and wildtype


,Comparison,P_Value,Significant
Mutated/Wildtype,TP53_proteomics,6.030871e-11,True


In [31]:
# Repeat Step 2 & 3 - binary column 
prot_miss_trunc = get_missense_truncation_comparison(brain, prot_and_mutations)
prot_wildtype = TP53_prot_mut_wt.loc[TP53_prot_mut_wt['binary_mutations'] == 'Wildtype'] # select all wildtype 
TP53_prot_miss_trunc_wildtype = prot_miss_trunc.append(prot_wildtype, sort=False) # use for graph

# compare truncation and wildtype
prot_trunc_wt = TP53_prot_miss_trunc_wildtype.loc[TP53_prot_miss_trunc_wildtype['binary_mutations'] != 'Missense']
print("Doing t-test comparison for truncation and wildtype")
prot_results_trunc = wrap_ttest_return_all(prot_trunc_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results_trunc = prot_results_trunc.rename(index = {0: "Truncation/Wildtype"})

TP53_prot_results_miss_trunc = prot_results_trunc
TP53_prot_results_miss_trunc
TP53_prot_miss_trunc_wildtype

Doing t-test comparison for truncation and wildtype


/Users/Lindsey/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Name,TP53_proteomics,binary_mutations
Sample_ID,,
S001,0.044573,Missense
S002,0.117856,Missense
S004,1.832557,Missense
S009,0.762037,Missense
S022,-0.089556,Missense
...,...,...
S098,NaN,Wildtype
S100,-0.343235,Wildtype
S101,NaN,Wildtype


# Visualize 

In [32]:
# Create Dataframe 

RB1_cis_mut = RB1_prot_mut_wt.assign(Gene= "RB1")
CDK2_trans_mut = CDK2_prot_mut_wt.assign(Gene = "CDK2")
CDK6_trans_mut = CDK6_prot_mut_wt.assign(Gene = "CDK6")
CDKN2A_trans_mut = CDKN2A_prot_mut_wt.assign(Gene = "CDKN2A")
TFDP1_trans_mut = TFDP1_prot_mut_wt.assign(Gene = "TFDP1")
MORF4L2_trans_mut = MORF4L2_prot_mut_wt.assign(Gene = "MORF4L2")
CCND2_trans_mut = CCND2_prot_miss_trunc_wildtype.assign(Gene = "CCND2")


RB1_all = pd.concat([RB1_cis_mut, CDK2_trans_mut, CDK6_trans_mut, CDKN2A_trans_mut, TFDP1_trans_mut, MORF4L2_trans_mut,CCND2_trans_mut])
RB1_all = pd.merge(RB1_all, TP53_prot_miss_trunc_wildtype, on = "Sample_ID")
RB1_alldf = pd.melt(RB1_all, id_vars = ["Gene", 'binary_mutations_x', 'binary_mutations_y'], value_name = "omics")
RB1_alldf = RB1_alldf.rename(columns = {"binary_mutations_y": "TP53_mutation"})
RB1_alldf

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,Gene,binary_mutations_x,TP53_mutation,variable,omics
0,RB1,Mutated,Missense,CCND2_proteomics,NaN
1,CDK2,Mutated,Missense,CCND2_proteomics,NaN
2,CDK6,Mutated,Missense,CCND2_proteomics,NaN
3,CDKN2A,Mutated,Missense,CCND2_proteomics,NaN
4,TFDP1,Mutated,Missense,CCND2_proteomics,NaN
...,...,...,...,...,...
5539,CDK6,Wildtype,Missense,TP53_proteomics,NaN
5540,CDKN2A,Wildtype,Missense,TP53_proteomics,NaN
5541,TFDP1,Wildtype,Missense,TP53_proteomics,NaN
5542,MORF4L2,Wildtype,Missense,TP53_proteomics,NaN


merge TP53 mutation type to label graph 

In [33]:
# Visualize
plt.rcParams['figure.figsize']=(14,8) #size of plot
boxplot = sns.boxplot(x='Gene', y="value", hue = 'binary_mutations', 
                         hue_order = ['Wildtype', 'Truncation'], data = RB1_alldf, showfliers = False)    
boxplot.set_title("GBM RB1 Mutation Type effect on Proteomics Abundance")
boxplot = sns.stripplot(x='Gene', y="value", hue = 'binary_mutations', data = RB1_alldf, jitter = True, 
                           color = ".3", hue_order = ['Wildtype', 'Truncation'], dodge = True)
boxplot.set(xlabel = "interacting genes", ylabel = 'value')

boxplot.set(xlabel = "interacting genes", ylabel = 'value')

plt.show()
plt.clf()
plt.close()


ValueError: Could not interpret input 'value'

In [ ]:
# Visualize
plt.rcParams['figure.figsize']=(14,8) #size of plot
boxplot = sns.boxplot(x='Gene', y="value", hue = 'binary_mutations', 
                         hue_order = ['Wildtype', 'Truncation'], data = RB1_alldf, showfliers = False)    
boxplot.set_title("GBM RB1 Mutation Type effect on Proteomics Abundance")
boxplot = sns.stripplot(x='Gene', y="value", hue = 'binary_mutations', data = RB1_alldf, jitter = True, 
                           color = ".3", hue_order = ['Wildtype', 'Truncation'], dodge = True)
boxplot.set(xlabel = "interacting genes", ylabel = 'value')

plt.show()

plt.clf()
plt.close()

In [ ]:
# Create Dataframe 

RB1_cis_mut = RB1_prot_miss_trunc_wildtype.assign(Gene= "RB1")
CDK2_trans_mut = CDK2_prot_miss_trunc_wildtype.assign(Gene = "CDK2")
CDK6_trans_mut = CDK6_prot_miss_trunc_wildtype.assign(Gene = "CDK6")
CDKN2A_trans_mut = CDKN2A_prot_miss_trunc_wildtype.assign(Gene = "CDKN2A")
TFDP1_trans_mut = TFDP1_prot_miss_trunc_wildtype.assign(Gene = "TFDP1")
CCND2_trans_mut = CCND2_prot_miss_trunc_wildtype.assign(Gene = "CCND2")
TP53_trans_mut = TP53_prot_miss_trunc_wildtype.assign(Gene = "TP53")

RB1_all = pd.concat([RB1_cis_mut, CDK2_trans_mut, CDK6_trans_mut, CDKN2A_trans_mut,CCND2_trans_mut, TFDP1_trans_mut])
RB1_alldf = pd.melt(RB1_all, id_vars = ["Gene", 'binary_mutations'])
RB1_all

In [ ]:
# Visualize
plt.rcParams['figure.figsize']=(14,8) #size of plot
boxplot = sns.boxplot(x='Gene', y="value", hue = 'binary_mutations', 
                         hue_order = ['Wildtype', 'Truncation'], data = RB1_alldf, showfliers = False)    
boxplot.set_title("GBM RB1 Mutation Type effect on Proteomics Abundance")
boxplot = sns.stripplot(x='Gene', y="value", hue = 'binary_mutations', data = RB1_alldf, jitter = True, 
                           color = ".3", hue_order = ['Wildtype', 'Truncation'], dodge = True)
boxplot.set(xlabel = "interacting genes", ylabel = 'value')

plt.show()

plt.clf()
plt.close()

# Extra checks 

looking for E2F1 in transcriptomics and proteomics 

In [ ]:
#num_freq_mut_genes = len(freq_mut)
num_freq_mut_genes = 20
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest - drop 
transrcpt_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'transcriptomics', omics_genes = "E2F1")
reduced_transcript_mutations = brain.reduce_multiindex(transrcpt_mutations, levels_to_drop=1) #single col labels

# Step 2 & 3 - Get binary column and format
transcript_mut_wt = format_mutated_wt_comparison(reduced_transcript_mutations)
transcript_mut_wt

In [ ]:
# Step 4 - T-test
trans_col_list = ['E2F1_transcriptomics']
trans_col = 'E2F1_transcriptomics'

print("Doing t-test comparison for mutation status")
transcript_results = wrap_ttest_return_all(transcript_mut_wt, 'binary_mutations', trans_col_list, num_freq_mut_genes)
transcript_results = transcript_results.rename(index = {0: "Mutated/Wildtype"})
transcript_results

In [ ]:

# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "E2F1") # single col level


# Step 2 & 3 - Get binary column and format
prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
prot_mut_wt.loc[prot_mut_wt ["binary_mutations"] == "Mutated"]

In [ ]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["E2F1"+'_proteomics']
prot_col = "E2F1"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

In [ ]:
df = brain.get_proteomics()
E2_cols = [col for col in df.columns if "CCNA" in col]
print (E2_cols)

In [ ]:
df["E2F3"]

looking for CCNE1

In [ ]:
#num_freq_mut_genes = len(freq_mut)
num_freq_mut_genes = 20
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest - drop 
transrcpt_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'transcriptomics', omics_genes = "CCNA2")
reduced_transcript_mutations = brain.reduce_multiindex(transrcpt_mutations, levels_to_drop=1) #single col labels

# Step 2 & 3 - Get binary column and format
transcript_mut_wt = format_mutated_wt_comparison(reduced_transcript_mutations)
transcript_mut_wt

In [ ]:
# Step 4 - T-test
trans_col_list = ['CCNA2_transcriptomics']
trans_col = 'CCNA2_transcriptomics'

print("Doing t-test comparison for mutation status")
transcript_results = wrap_ttest_return_all(transcript_mut_wt, 'binary_mutations', trans_col_list, num_freq_mut_genes)
transcript_results = transcript_results.rename(index = {0: "Mutated/Wildtype"})
transcript_results

In [ ]:

# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "CCNA2") # single col level


# Step 2 & 3 - Get binary column and format
prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
prot_mut_wt.loc[prot_mut_wt ["binary_mutations"] == "Mutated"]

In [ ]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CCNA2"+'_proteomics']
prot_col = "CCNA2"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

In [ ]:
df = brain.get_proteomics()
CCN_cols = [col for col in df.columns if "BAK" in col]
print (CCN_cols)


In [ ]:
df.columns.get_loc("CCND1")
df.iloc[0:4,1465:1480]

could not find E2F1 or CCNE1 in proteomics, but were significant in transcriptomics. 

In [ ]:
#num_freq_mut_genes = len(freq_mut)
num_freq_mut_genes = 20
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest - drop 
transrcpt_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'transcriptomics', omics_genes = "CDK2")
reduced_transcript_mutations = brain.reduce_multiindex(transrcpt_mutations, levels_to_drop=1) #single col labels

# Step 2 & 3 - Get binary column and format
transcript_mut_wt = format_mutated_wt_comparison(reduced_transcript_mutations)
transcript_mut_wt

In [ ]:
#num_freq_mut_genes = len(freq_mut) 
num_freq_mut_genes = 20
prot_col_list = ["CDK4"+'_proteomics']
prot_col = "CDK4"+"_proteomics"

# Step 4 - T-test comparing means of mutated vs wildtype effect on cis omics
print("Doing t-test comparison for mutated and wildtype")
prot_results = wrap_ttest_return_all(CDK2_prot_mut_wt, 'binary_mutations', prot_col_list, num_freq_mut_genes)
prot_results = prot_results.rename(index = {0: "Mutated/Wildtype"})
prot_results

# BIRC5

In [ ]:
#num_freq_mut_genes = len(freq_mut)
num_freq_mut_genes = 20
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest - drop 
transrcpt_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'transcriptomics', omics_genes = "BIRC5")
reduced_transcript_mutations = brain.reduce_multiindex(transrcpt_mutations, levels_to_drop=1) #single col labels

# Step 2 & 3 - Get binary column and format
transcript_mut_wt = format_mutated_wt_comparison(reduced_transcript_mutations)
transcript_mut_wt

In [ ]:
# Step 4 - T-test
trans_col_list = ['BIRC5_transcriptomics']
trans_col = 'BIRC5_transcriptomics'

print("Doing t-test comparison for mutation status")
transcript_results = wrap_ttest_return_all(transcript_mut_wt, 'binary_mutations', trans_col_list, num_freq_mut_genes)
transcript_results = transcript_results.rename(index = {0: "Mutated/Wildtype"})
transcript_results

In [ ]:

# Step 1 - Create dataframe in order to do comparisons with wrap_ttest  
prot_and_mutations = brain.join_omics_to_mutations(
    mutations_genes = gene, omics_df_name = 'proteomics', omics_genes = "BIRC5") # single col level


# Step 2 & 3 - Get binary column and format
prot_mut_wt = format_mutated_wt_comparison(prot_and_mutations)
prot_mut_wt.loc[prot_mut_wt ["binary_mutations"] == "Mutated"]